### Load Data

In [1]:
import pandas as pd

courses = pd.read_csv('./Data/Coursera_courses.csv')
users = pd.read_csv('./Data/Coursera_reviews.csv')
courses_metadata = pd.read_csv('./Data/CourseraDataset-Unclean.csv')
job_skills = pd.read_csv('./Data/job_skills.csv')
job_metadata = pd.read_csv('./Data/linkedin_job_postings.csv')

### Remove Duplicate Courses

In [2]:
courses_metadata.rename(columns={'Course Title': 'name'}, inplace = True)
courses_metadata.drop_duplicates(subset=['name'], inplace=True)

### Merge Metadata regarding Courses with Users

In [3]:
temp = pd.merge(users, courses, on="course_id", how="inner")
courses_data = pd.merge(temp, courses_metadata, on="name", how="inner")
#temp.head()
#courses_data.head()

### Filter Essential Columns in Courses Data

In [51]:
final_courses = courses_data[['reviews', 'reviewers', 'rating', 'name', 'institution', 'Rating', 'Level', 'Duration', 'Review', 'Skill gain', 'Instructor']]
final_courses = final_courses.rename(columns={ 'Rating': 'Overall Ratings', 'Review': 'Num of Reviews'})
#final_courses.head()

,reviews,reviewers,rating,name,institution,Overall Ratings,Level,Duration,Num of Reviews,Skill gain,Instructor
0,"Pretty dry, but I was able to pass with just t...",By Robert S,4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,4.5,NaN,Approx. 3 hours to complete,191 reviews,[],['Google']
1,would be a better experience if the video and ...,By Gabriel E R,4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,4.5,NaN,Approx. 3 hours to complete,191 reviews,[],['Google']
2,Information was perfect! The program itself wa...,By Jacob D,4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,4.5,NaN,Approx. 3 hours to complete,191 reviews,[],['Google']
3,A few grammatical mistakes on test made me do ...,By Dale B,4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,4.5,NaN,Approx. 3 hours to complete,191 reviews,[],['Google']
4,Excellent course and the training provided was...,By Sean G,4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,4.5,NaN,Approx. 3 hours to complete,191 reviews,[],['Google']


### Merge Datasets regarding Jobs

In [5]:
jobs_data = pd.merge(job_skills, job_metadata, on="job_link", how="inner")
#jobs_data.head()

### Filter Essential Columns in Jobs Data

In [6]:
final_jobs = jobs_data[['job_skills', 'job_title', 'search_position', 'job_level']]
#final_jobs.head()
# unique_values = final_jobs['search_position'].unique()
#print(unique_values.tolist())

### Preprocess 
1) Prices
2) Remove "By' from reviewer
3) Fill in null values in level
4) get the number in duration, 
5) get the number in review, 
6) remove the list of skill gain
7) remove the list of instructor


In [52]:
final_courses['reviewers'] = final_courses['reviewers'].apply(lambda x: x[3:])
final_courses['Level'].fillna('None', inplace=True)
final_courses['Duration'] = final_courses['Duration'].str.extract('(\d+)').astype(int)
final_courses['Num of Reviews'] = final_courses['Num of Reviews'].str.extract('(\d+)').astype(int)
final_courses['Skill gain'] = final_courses['Skill gain'].apply(lambda x: x[1:-1])
final_courses['Instructor'] = final_courses['Instructor'].apply(lambda x: x[2:-2])
final_courses['Skill gain'] = final_courses['Skill gain'].replace('', 'No Specific Skills')

# final_courses.head()

### Handle Null Values

In [65]:
# print(final_jobs.isnull().sum())
# print(final_courses.isnull().sum())

# # Cant do anything with empty job_skills column; got more than enough data so dropna
# # reviews column, are we using reviews to users to do collaborative filtering?
final_jobs = final_jobs.dropna(subset=['job_skills'])
# print(final_jobs.isnull().sum())